In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
42055,Япония,2020-08-14,53818,35498,1080,1347,-693,7
42056,Япония,2020-08-15,55051,39439,1093,1233,3941,13
42057,Япония,2020-08-16,56074,40560,1103,1023,1121,10
42058,Япония,2020-08-17,56717,41591,1112,643,1031,9
42059,Япония,2020-08-18,57636,42965,1135,919,1374,23
42060,Япония,2020-08-19,58728,43786,1148,1092,821,13
42061,Япония,2020-08-20,59900,45110,1157,1172,1324,9
42062,Япония,2020-08-21,60949,46264,1175,1049,1154,18
42063,Япония,2020-08-22,61916,47418,1179,967,1154,4
42064,Япония,2020-08-23,62658,48319,1188,742,901,9


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
215,Весь мир,2020-08-24,23643821,15337252,812983,226334,200049,4335
216,Весь мир,2020-08-25,23885837,15570191,819379,242016,232939,6396
217,Весь мир,2020-08-26,24173588,15794441,825661,287751,224250,6282
218,Весь мир,2020-08-27,24452689,15995995,831547,279101,201554,5886
219,Весь мир,2020-08-28,24734448,16196934,837080,281759,200939,5533
220,Весь мир,2020-08-29,24996456,16408559,842452,262008,211625,5372
221,Весь мир,2020-08-30,25222709,16616995,846353,226253,208436,3901
222,Весь мир,2020-08-31,25484767,16818436,850535,262058,201441,4182
223,Весь мир,2020-09-01,25749642,17073236,857015,264875,254800,6480
224,Весь мир,2020-09-02,26031410,17291874,863028,281768,218638,6013


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2020-09-02


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'Доминика'), (51, 'Доминиканская Республика'), (52, 'Египет'), (53, 'Замбия'), (54,

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
42295,2020-08-29,67353,54883,1271,854,1192,20,1.179,1.050,0,0,Япония,11199,776.714286,878.714286,12194.857143,12970.071429,8,-358
42296,2020-08-30,67958,55700,1285,605,817,14,1.110,1.051,0,0,Япония,10973,757.142857,848.857143,11883.714286,12724.500000,-249,-226
42297,2020-08-31,68396,56338,1298,438,638,13,0.916,1.049,0,0,Япония,10760,748.285714,834.214286,11582.571429,12492.071429,-167,-213
42298,2020-09-01,69023,57503,1313,627,1165,15,0.755,1.046,0,0,Япония,10207,733.571429,813.357143,11244.000000,12254.285714,189,-553
42299,2020-09-02,69619,58108,1327,596,605,14,0.654,1.041,0,0,Япония,10184,691.428571,777.928571,10964.285714,11996.428571,-31,-23


In [6]:
# Добавляем колонки расчитанные сколько доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-09-01   25749642        7819391
2020-09-02   26031410        7876508
42300
(42300, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
29005,2020-08-14,910778,Россия,4.294406,2.424087
29006,2020-08-15,915808,Россия,4.268143,2.378490
29007,2020-08-16,920719,Россия,4.248941,2.403870
29008,2020-08-17,925558,Россия,4.230451,2.428974
29009,2020-08-18,930276,Россия,4.202799,2.395846
29010,2020-08-19,935066,Россия,4.172872,2.351057
29011,2020-08-20,939833,Россия,4.144738,2.314787
29012,2020-08-21,944671,Россия,4.116922,2.271603
29013,2020-08-22,949531,Россия,4.092711,2.237980
29014,2020-08-23,954328,Россия,4.075279,2.262972


In [8]:
X.to_csv('W.csv',index=False)